In [3]:
import numpy as np
import matplotlib.pyplot as plt
import inspect

In [2]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

print(sys.path)


['/Users/gali/anaconda3/envs/LIMEtf/lib/python310.zip', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/lib-dynload', '', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/setuptools/_vendor', '../', '../']


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [4]:
x_test_original, y_test_original,x_train, y_train = ReadTS('ChlorineConcentration')
y_train = y_train - 1
y_test_original = y_test_original - 1
print(x_train.shape)
print(y_train)

(3840, 166, 1)
[1. 1. 0. ... 2. 2. 2.]


In [5]:
model = make_CNN_model(x_train.shape[1:],num_classes = 3)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=100, batch_size=32)

Epoch 1/100


2024-10-30 20:16:19.685876: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


96/96 [==============================] - 1s 9ms/step - loss: 1.0096 - sparse_categorical_accuracy: 0.5286 - val_loss: 1.0969 - val_sparse_categorical_accuracy: 0.2578
Epoch 2/100
96/96 [==============================] - 1s 8ms/step - loss: 0.9811 - sparse_categorical_accuracy: 0.5586 - val_loss: 1.1046 - val_sparse_categorical_accuracy: 0.3568
Epoch 3/100
96/96 [==============================] - 1s 9ms/step - loss: 0.9733 - sparse_categorical_accuracy: 0.5612 - val_loss: 1.0663 - val_sparse_categorical_accuracy: 0.4453
Epoch 4/100
96/96 [==============================] - 1s 9ms/step - loss: 0.9626 - sparse_categorical_accuracy: 0.5632 - val_loss: 0.9954 - val_sparse_categorical_accuracy: 0.5560
Epoch 5/100
96/96 [==============================] - 1s 9ms/step - loss: 0.9577 - sparse_categorical_accuracy: 0.5632 - val_loss: 1.0807 - val_sparse_categorical_accuracy: 0.3893
Epoch 6/100
96/96 [==============================] - 1s 9ms/step - loss: 0.9549 - sparse_categorical_accuracy: 0.5674

In [6]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

15/15 [==============================] - 0s 3ms/step - loss: 2.4619 - sparse_categorical_accuracy: 0.5931
Test accuracy 0.5931477546691895
Test loss 2.4619076251983643


### Create perturbed test sets and calculate decrease in accuracy

In [63]:
print(y_test_original.shape)

(3840,)


In [50]:
print(inspect.getsource(perturb))
print(inspect.signature(perturb))

def perturb(perturbation_strategy, ts, index0, index1, global_ts = []):
    if perturbation_strategy == 'RBP':
        return RBPIndividual(ts, index0, index1)
    if perturbation_strategy == 'zero':
        return zeroPerturb(ts, index0, index1)
    if perturbation_strategy == 'noise':
        return noisePerturb(ts, index0, index1)
    if perturbation_strategy == 'blur':
        return blurPerturb(ts, index0, index1)
    if perturbation_strategy == 'RBP1':
        return RBPIndividualNew1(global_ts, ts, index0, index1)    
    if perturbation_strategy == 'RBP2':
        return RBPIndividualNew2(global_ts, ts, index0, index1)    

(perturbation_strategy, ts, index0, index1, global_ts=[])


In [12]:
predictions_original = model.predict(x_test_original)
perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])

x_test_rbp = np.asarray([perturb('RBP',x,60,160) for x in perturb_test_set])
x_test_rbp = np.expand_dims(x_test_rbp, axis=-1)
predictions_rbp = model.predict(x_test_rbp)

pert1 = perturb('RBP1',perturb_test_set[0],60,160, global_ts = perturb_test_set)
x_test_new1 = np.asarray([pert1 for x in perturb_test_set])
x_test_new1 = np.expand_dims(x_test_new1, axis=-1)
predictions_new1 = model.predict(x_test_new1)

#x_test_new2 = np.asarray([perturb('RBP2',x,400,500, global_ts = perturb_test_set) for x in perturb_test_set])
#x_test_new2 = np.expand_dims(x_test_new2, axis=-1)
#predictions_new2 = model.predict(x_test_new2)

x_test_zero = np.asarray([perturb('zero',x,60,160) for x in perturb_test_set])
x_test_zero = np.expand_dims(x_test_zero, axis=-1)
predictions_zero = model.predict(x_test_zero)

x_test_noise = np.asarray([perturb('noise',x,60,160) for x in perturb_test_set])
x_test_noise = np.expand_dims(x_test_noise, axis=-1)
predictions_noise = model.predict(x_test_noise)

x_test_blur = np.asarray([perturb('blur',x,60,160) for x in perturb_test_set])
x_test_blur = np.expand_dims(x_test_blur, axis=-1)
predictions_blur = model.predict(x_test_blur)

15/15 [==============================] - 0s 2ms/step


In [13]:
def accuracy(predictions, labels):
    hits = 0
    for i in range(0,len(predictions)):
        if predictions[i] == labels[i]:
            hits += 1
    return hits/len(predictions)

In [15]:
original_accuracy = accuracy(np.argmax(predictions_original,axis=1), y_test_original)
rbp_accuracy = accuracy(np.argmax(predictions_rbp,axis=1), y_test_original)
new1_accuracy = accuracy(np.argmax(predictions_new1,axis=1), y_test_original)
#new2_accuracy = accuracy(np.argmax(predictions_new2,axis=1), y_test_original)
zero_accuracy = accuracy(np.argmax(predictions_zero,axis=1), y_test_original)
noise_accuracy = accuracy(np.argmax(predictions_noise,axis=1), y_test_original)
blur_accuracy = accuracy(np.argmax(predictions_blur,axis=1), y_test_original)

In [17]:
print(" Original Accuracy: " + str(original_accuracy) + "\n"+
    " RBP Accuracy: " + str(rbp_accuracy) + "\n"+
    " RBP1 Accuracy: " + str(new1_accuracy) + "\n"+
    #" RBP2 Accuracy: " + str(new2_accuracy) + "\n"+
      
    " Zero Accuracy: " + str(zero_accuracy) + "\n"+
    " Noise Accuracy: " + str(noise_accuracy) + "\n"+
    " Blur Accuracy: " + str(blur_accuracy) + "\n")

 Original Accuracy: 0.5931477516059958
 RBP Accuracy: 0.4004282655246253
 RBP1 Accuracy: 0.24411134903640258
 Zero Accuracy: 0.278372591006424
 Noise Accuracy: 0.24411134903640258
 Blur Accuracy: 0.5182012847965739

